### Information processing primitives for the Joglekar & Wang rate model


The model ([Joglekar et. al., 2018](https://www.sciencedirect.com/science/article/pii/S0896627318301521)) is constructed using structural data from the monkey ([Markov et. al., 2013](https://academic.oup.com/cercor/article/24/1/17/272931)), and it is used to study mechanisms of propagation in the brain network via a mechanism called **global balance amplification (GBA)**, which consists of increasing long range excitatory to excitatory connections, and local recurrent inhibitions in order to maintain stability. Here, weak and strong GBA will be used as proxies for reduced and increased attention, respectively.

In [ ]:
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/IPP_WANG")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from frites.conn import conn_te
from tqdm import tqdm

from src.infodyn.conn_pid import conn_pid

In [ ]:
fig = plt.figure(figsize=(4, 2), dpi=300)


def plot_png(ax, figname):
    png = plt.imread(figname)
    plt.sca(ax)
    im = plt.imshow(png, interpolation="none")
    plt.axis("off")
    pad = 10
    plt.xlim(-pad, png.shape[1] + pad)
    plt.ylim(png.shape[0] + pad, -pad)


gs0 = fig.add_gridspec(
    nrows=1,
    ncols=2,
    left=0.05,
    right=0.95,
    hspace=0.05,
    bottom=0.05,
    top=0.95,
)

axs0 = [plt.subplot(gs0[i]) for i in range(2)]

for i in range(2):
    plot_png(axs0[i], f"figures/circuit{i + 1}.png")
    if i == 0:
        plt.title("Weak GBA")
    else:
        plt.title("Strong GBA")

plt.savefig("figures/circuitGBA.pdf")

In [ ]:
protocol = 2

In [ ]:
rates = xr.load_dataarray(f"../data/protocol{protocol}.nc")[..., ::10]

In [ ]:
if protocol == 2:
    rates = rates.sel(times=slice(3000, 7000))

#### Firing rate average over trials

Here, we simulated two types of trials: fixation (F) and task (T). Those are used to emulate working memory tasks, were in task trials a stimulus is presented during a cue period, and a match period that are separated by a delay (or memory) period. In fixation trials, no stimulus is presented, therefore when computing information primitives, we use the presence or absence of stimulus as stimulus label (thus, there is no computation involve stimulus identity, only stimulus presence).


For the task trials, the stimulus is presented from 1000 to 1500 ms, and again from 3500 to 3800 ms. In total, 1000 task, and a 1000 fixation trials were simulated for each condition, strong and weak GBA. Hence, 4000 trials in total. The total simulation time was 7000 ms.

In [ ]:
rois, times = rates.roi.data, rates.times.data
times = (times - 4500) / 1000

In [ ]:
if protocol == 0:
    tticks = np.round(np.array([1.0, 1.5, 3.5, 3.8]) - 1.0, 2)
elif protocol == 1:
    tticks = np.round(np.array([0.0, 1.5, 2.0, 3.0]) - 1.5, 2)
elif protocol == 2:
    tticks = np.round(np.array([4.5, 5., 6, 7.5]) - 4.5, 2)
    
tidx = [np.abs(times - t).argmin() for t in tticks]

In [ ]:
def get_time_idx(times):
    if protocol == 0:
        tticks = np.round(np.array([1.0, 1.5, 3.5, 3.8]) - 1.0, 2)
    elif protocol == 1:
        tticks = np.round(np.array([0.0, 1.5, 2.0, 3.0]) - 1.5, 2)
    elif protocol == 2:
        tticks = np.round(np.array([4.5, 5., 6, 7.5]) - 4.5, 2)

    tidx = [np.abs(times - t).argmin() for t in tticks]  
    
    return tidx, tticks

##### Fixation

In [ ]:
rates_avg_f = rates.sel(trials=0).mean("trials")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)

plt.imshow(rates_avg_f.sel(gba="weak"), aspect="auto", origin="lower", cmap="Greys")
plt.xticks(tidx, tticks)
plt.yticks(range(rates_avg_f.sizes["roi"]), rates_avg_f.roi.data)
plt.title("Weak GBA")
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Firing Rate [Hz]", rotation=270, labelpad=12)
plt.savefig("figures/fr_fixation_weakGBA.eps")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)

plt.imshow(rates_avg_f.sel(gba="strong"), aspect="auto", origin="lower", cmap="Greys")
plt.xticks(tidx, tticks)
plt.yticks(range(rates_avg_f.sizes["roi"]), rates_avg_f.roi.data)
plt.title("strong GBA")
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Firing Rate [Hz]", rotation=270, labelpad=12)
plt.savefig(f"figures/fr_fixation_strongGBA_p_{protocol}.eps")

Time-course (x-axis) of the avergae firing rate of each cortical area (y-axis) in the model, for **fixation** trials. 

##### Task

In [ ]:
rates_avg_t = rates.sel(trials=1).mean("trials")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)

plt.imshow(rates_avg_t.sel(gba="weak"), aspect="auto", origin="lower", cmap="Greys")
plt.xticks(tidx, tticks)
plt.yticks(range(rates_avg_f.sizes["roi"]), rates_avg_f.roi.data)
plt.title("Weak GBA")
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Firing Rate [Hz]", rotation=270, labelpad=12)
plt.savefig(f"figures/fr_task_weakGBA_p_{protocol}.eps")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)

plt.imshow(rates_avg_t.sel(gba="strong"), aspect="auto", origin="lower", cmap="Greys")
plt.xticks(tidx, tticks)
plt.yticks(range(rates_avg_f.sizes["roi"]), rates_avg_f.roi.data)
plt.title("strong GBA")
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Firing Rate [Hz]", rotation=270, labelpad=12)
plt.savefig(f"figures/fr_task_strongGBA_p_{protocol}.eps")

Time-course (x-axis) of the avergae firing rate of each cortical area (y-axis) in the model, for **task** trials. 

#### Partial Information Decomposition

In [ ]:
pairs = np.stack(([0] * 28, range(1, 29)), axis=1)

In [ ]:
pid_weak = conn_pid(rates.sel(gba="weak"), "trials", "roi", "times", "cd", pairs=pairs)

In [ ]:
pid_strong = conn_pid(
    rates.sel(gba="strong"), "trials", "roi", "times", "cd", pairs=pairs
)

#### Mutual Information MI(RATE, STIMULUS PRESENCE)

In [ ]:
plt.figure(figsize=(15, 8), dpi=300)
plt.subplot(1, 2, 1)
plt.imshow(pid_weak[0], aspect="auto", cmap="Greys", origin="lower")
plt.yticks(range(rates_avg_t.sizes["roi"]), rates_avg_t.roi.data)
plt.title("Weak GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(pid_strong[0], aspect="auto", cmap="Greys", origin="lower")
plt.yticks(range(rates_avg_t.sizes["roi"]), rates_avg_t.roi.data)
plt.title("Strong GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
plt.colorbar()

Mutual information between firing rate and stimulus presence (fixation vs. task trials), for each cortical area (y-axis) and time (x-axis).

#### Sinergy

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(pid_weak[4], aspect="auto", cmap="Greys", origin="lower", vmin=0)
plt.yticks(range(pid_weak[4].sizes["roi"]), pid_weak[2].roi.data)
plt.title("Weak GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Synergy", rotation=270, labelpad=12)
plt.savefig(f"figures/synergy_space_time_weak_p_{protocol}.eps")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(pid_strong[4], aspect="auto", cmap="Greys", origin="lower", vmin=0)
plt.yticks(range(pid_weak[4].sizes["roi"]), pid_weak[2].roi.data)
plt.title("Strong GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("Synergy", rotation=270, labelpad=12)
plt.savefig(f"figures/synergy_space_time_strong_p_{protocol}.eps")

Same as the plot above, however without thaking the average over areas (y-axis).

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)

syn_w = pid_weak[-1].mean("times")
syn_s = pid_strong[-1].mean("times")

# Compute pie slices
N = 28
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

plt.subplot(polar=True)

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois[1:]))


# Plot actual sales graph
plt.plot(theta, syn_w, "r")
plt.fill(theta, syn_w, "r", alpha=0.1)
plt.plot(theta, syn_s, "b")
plt.fill(theta, syn_s, "b", alpha=0.1)

plt.savefig(f"figures/synergy_time_averaged_p_{protocol}.svg")

Total synergy (in this case averge over time), distribution for each cortical area for weak (red) and strong (blue) GBA.

#### Active information storage

In [ ]:
from frites.core import gcmi_nd_cc

In [ ]:
rois = rates.roi.data

In [ ]:
AISw = []

for roi in tqdm(rois):
    AISw += [
        gcmi_nd_cc(
            rates.sel(gba="weak", roi=roi)[..., :-40].data,
            rates.sel(gba="weak", roi=roi)[..., 40:].data,
            traxis=0,
            mvaxis=None,
        )
    ]

AISw = xr.DataArray(AISw, dims=("roi", "times"), coords=(rois, rates.times[:-40]))

In [ ]:
AISs = []

for roi in tqdm(rois):
    AISs += [
        gcmi_nd_cc(
            rates.sel(gba="strong", roi=roi)[..., :-40].data,
            rates.sel(gba="strong", roi=roi)[..., 40:].data,
            traxis=0,
            mvaxis=None,
        )
    ]

AISs = xr.DataArray(AISs, dims=("roi", "times"), coords=(rois, rates.times[:-40]))

In [ ]:
AIS = xr.concat([AISw, AISs], "gba")

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)

plt.imshow(AIS.isel(gba=0), aspect="auto", cmap="Greys", origin="lower")
plt.title("Weak GBA")
plt.xlabel("Time [a.u]")
cbar = plt.colorbar()
cbar.set_label("AIS", rotation=270, labelpad=12)
plt.savefig("figures/AIS_space_time_weak.eps")

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)

plt.imshow(AIS.isel(gba=1), aspect="auto", cmap="Greys", origin="lower")
plt.title("Strong GBA")
plt.xlabel("Time [a.u]")
cbar = plt.colorbar()
cbar.set_label("AIS", rotation=270, labelpad=12)
plt.savefig(f"figures/AIS_space_time_strong_p_{protocol}.eps")

Active information storage time-course (x-axis) for each cortical area (y-axis) in the model. For the weak (left) and strong GBA (right) states.

In [ ]:
ais = AIS.mean("times")

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)


# Compute pie slices
N = 29
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

plt.subplot(polar=True)

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois))


# Plot actual sales graph
plt.plot(theta, ais.sel(gba=0), "r")
plt.fill(theta, ais.sel(gba=0), "r", alpha=0.1)
plt.plot(theta, ais.sel(gba=1), "b")
plt.fill(theta, ais.sel(gba=1), "b", alpha=0.1)

plt.savefig(f"figures/AIS_time_averaged_p_{protocol}.svg")

Total active information storage (in this case averge over time), distribution for each cortical area for weak (red) and strong (blue) GBA.

#### Transfer entropy

In [ ]:
TEfb_w = conn_te(
    rates.sel(gba="weak"), "times", "roi", max_delay=40, n_jobs=30, pairs=pairs[:, ::-1]
)
TEff_w = conn_te(
    rates.sel(gba="weak"), "times", "roi", max_delay=40, n_jobs=30, pairs=pairs
)

In [ ]:
TEfb_s = conn_te(
    rates.sel(gba="strong"),
    "times",
    "roi",
    max_delay=40,
    n_jobs=30,
    pairs=pairs[:, ::-1],
)
TEff_s = conn_te(
    rates.sel(gba="strong"), "times", "roi", max_delay=40, n_jobs=30, pairs=pairs
)

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(TEff_w, aspect="auto", cmap="Greys", origin="lower", vmin=0)
plt.yticks(range(TEff_w.sizes["roi"]), TEff_w.roi.data)
plt.title("Weak GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("FF Transfer Entropy [bits]", rotation=270, labelpad=12)
plt.savefig(f"figures/TE_feedforward_weakGBA_p_{protocol}.svg")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(TEfb_w, aspect="auto", cmap="Greys", origin="lower", vmin=0)
plt.yticks(range(TEfb_w.sizes["roi"]), TEfb_w.roi.data)
plt.title("Weak GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("FB Transfer Entropy [bits]", rotation=270, labelpad=12)
plt.savefig(f"figures/TE_feedback_weakGBA_p_{protocol}.svg")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(TEff_s, aspect="auto", cmap="Greys", origin="lower", vmin=0, vmax=0.3)
plt.yticks(range(TEff_s.sizes["roi"]), TEff_s.roi.data)
plt.title("Strong GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("FF Transfer Entropy [bits]", rotation=270, labelpad=12)
plt.savefig(f"figures/TE_feedforward_strongGBA_p_{protocol}.svg")

In [ ]:
plt.figure(figsize=(6, 8), dpi=300)
plt.imshow(TEfb_s, aspect="auto", cmap="Greys", origin="lower", vmin=0, vmax=0.3)
plt.yticks(range(TEfb_s.sizes["roi"]), TEfb_s.roi.data)
plt.title("Strong GBA")
plt.xticks(tidx, tticks)
plt.xlabel("Time [s]")
cbar = plt.colorbar()
cbar.set_label("FB Transfer Entropy [bits]", rotation=270, labelpad=12)
plt.savefig(f"figures/TE_feedback_strongGBA_p_{protocol}.svg")

In [ ]:
TEff_w_avg = TEff_w.mean("times")
TEff_s_avg = TEff_s.mean("times")

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)


# Compute pie slices
N = 28
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

plt.subplot(polar=True)

rois = rates.roi.data[1:]

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois))


# Plot actual sales graph
plt.plot(theta, TEff_w_avg, "r")
plt.fill(theta, TEff_w_avg, "r", alpha=0.1)
plt.plot(theta, TEff_s_avg, "b")
plt.fill(theta, TEff_s_avg, "b", alpha=0.1)

plt.title("FF")

plt.tight_layout()

plt.savefig(f"figures/TE_feedforward_time_averaged_p_{protocol}.svg")

In [ ]:
TEfb_w_avg = TEfb_w.mean("times")
TEfb_s_avg = TEfb_s.mean("times")

In [ ]:
rV1 = rates_avg_t.sel(gba="strong", roi="V1")
r24c = 4 * rates_avg_t.sel(gba="strong", roi="46d")
syn = 350 * pid_strong[4].sel(roi="46d-V1")

In [ ]:
fig, ax = plt.subplots()
rV1.plot(ax=ax)
r24c.plot(ax=ax)
plt.title("")
syn.plot(ax=ax, c="g")
ax.legend(["rate V1", "4 * rate 46d", "350 * syn"])

plt.title("")

In [ ]:
tau1 = rates_avg_t[..., 100:].sel(gba="strong").argmax("times")

In [ ]:
tau3 = rates_avg_t[..., 100:].sel(gba="strong", roi=["V1"]).argmax("times")

In [ ]:
tau2 = pid_strong[4][..., 100:].argmax("times")

In [ ]:
plt.step(x=range(28), y=np.abs(tau1[1:].data - tau2.data) * 0.1)
plt.step(x=range(28), y=np.abs(tau3.data - tau2.data) * 0.1)
plt.xticks(range(28), rois, rotation=90)
plt.legend(["V1 peak - syn peak", "rate peak of other areas (xaxis) - syn peak"])

In [ ]:
import os

import seaborn as sns
from scipy.io import loadmat

In [ ]:
path = f"/home/vinicius/storage1/projects/IPP_WANG/src/interareal/"

In [ ]:
data = np.load(os.path.join(path, "markov2014.npy"), allow_pickle=True).all()

In [ ]:
FLN, Distances, Hierarchy = data["FLN"], data["Distances"], data["Hierarchy"]

In [ ]:
x = Distances[0, 1:]
y = np.abs(tau1[1:].data - tau2.data) * 0.1
x = x[rois != "2"]
y = y[rois != "2"]
sns.regplot(x=x, y=y, n_boot=100, ci=None)
for i, r in enumerate(rois[rois != "2"]):
    plt.text(x[i], y[i], r)

plt.ylabel("Rate - Syn. peak time")
plt.xlabel(" D(V1, AREA) [mm]")

#### Stimulus specific storage

In [ ]:
from frites.core import gccmi_nd_ccnd
from frites.core import gcmi_nd_cc 

def fix_frites(func):  # noqa
    def _fix(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            return 0
    return _fix

gccmi_nd_ccnd = fix_frites(gccmi_nd_ccnd)
gcmi_nd_cc = fix_frites(gcmi_nd_cc)


def stimulus_specific_storage(roi=None, tau=1, gba="weak"):
    R = rates.sel(gba=gba, roi=roi)
    pos = R.trials.data
    AIS = gcmi_nd_cc(R[..., tau:].data, R[..., :-tau].data, mvaxis=None, traxis=0)
    AISc = gccmi_nd_ccnd(R[..., tau:].data, R[..., :-tau].data, pos, mvaxis=None, traxis=0)
    
    return AIS - AISc    

SSSw = [stimulus_specific_storage(roi=r, tau=40, gba="weak") for r in tqdm(rates.roi.data)]

SSSs = [stimulus_specific_storage(roi=r, tau=40, gba="strong") for r in tqdm(rates.roi.data)]


SSSw = np.stack(SSSw)
SSSs = np.stack(SSSs)

times = rates.times.data[:-40]
times = (times - 4500) / 1000

SSSw = xr.DataArray(SSSw, dims=("roi", "times"), coords=(rates.roi.data, times))
SSSs = xr.DataArray(SSSs, dims=("roi", "times"), coords=(rates.roi.data, times))

### Paper figure

In [ ]:
path = f"/home/vinicius/storage1/projects/IPP_WANG/src/interareal/"

area_names = [
    "V1",
    "V2",
    "V4",
    "DP",
    "MT",
    "8m",
    "5",
    "8l",
    "TEO",
    "2",
    "F1",
    "STPc",
    "7A",
    "46d",
    "10",
    "9/46v",
    "9/46d",
    "F5",
    "TEpd",
    "PBr",
    "7m",
    "7B",
    "F2",
    "STPi",
    "PROm",
    "F7",
    "8B",
    "STPr",
    "24c",
]
nareas = len(area_names)
iter_ = range(nareas)

data = np.load(os.path.join(path, "markov2014.npy"), allow_pickle=True).all()

FLN, Distances, Hierarchy = data["FLN"], data["Distances"], data["Hierarchy"]

In [ ]:
import matplotlib

import src.plot_utils as plot

In [ ]:
sel_rois = np.array(["V1", "V4", "8m", "8l", "TEO", "7A", "9/46d", "TEpd", "24c"])
sel_rois_syn = np.array(
    [f"V1-{r}" if r in ["V2", "V4"] else f"{r}-V1" for r in sel_rois[1:]]
)
sel_rois_teff = np.array([f"V1->{r}" for r in sel_rois[1:]])
sel_rois_tefb = np.array([f"{r}->V1" for r in sel_rois[1:]])

In [ ]:
fig = plt.figure(figsize=(8, 10), dpi=600)


gs0 = fig.add_gridspec(
    nrows=1,
    ncols=2,
    left=0.06,
    right=0.55,
    wspace=0.15,
    bottom=0.79,
    top=0.94,
)

gs1 = fig.add_gridspec(
    nrows=1,
    ncols=2,
    left=0.62,
    right=0.91,
    bottom=0.90,
    top=0.97,
)

gs2 = fig.add_gridspec(
    nrows=1,
    ncols=3,
    left=0.62,
    right=0.93,
    wspace=0.2,
    bottom=0.76,
    top=0.88,
    width_ratios=[1, 1, 0.05],
)

gs3 = fig.add_gridspec(
    nrows=1,
    ncols=8,
    left=0.06,
    right=0.95,
    wspace=0.5,
    bottom=0.58,
    top=0.7,
    width_ratios=[1, 0.05] * 4,
)

gs4 = fig.add_gridspec(
    nrows=1,
    ncols=4,
    left=0.06,
    right=0.95,
    bottom=0.3,
    top=0.55,
    hspace=0.5,
    wspace=0.5,
)

gs5 = fig.add_gridspec(
    nrows=1,
    ncols=2,
    left=0.35,
    right=0.95,
    bottom=0.05,
    top=0.28,
    wspace=0.3,
)

gs6 = fig.add_gridspec(
    nrows=1,
    ncols=1,
    left=0.05,
    right=0.3,
    bottom=0.05,
    top=0.26,
    wspace=0.3,
)

axs0 = [plt.subplot(gs0[i]) for i in range(2)]
axs1 = [plt.subplot(gs1[i]) for i in range(2)]
axs2 = [plt.subplot(gs2[i]) for i in range(3)]
axs3 = [plt.subplot(gs3[i]) for i in range(8)]
axs4 = [plt.subplot(gs4[i], polar=True) for i in range(4)]
axs5 = [plt.subplot(gs5[i]) for i in range(2)]
axs6 = [plt.subplot(gs6[i]) for i in range(1)]

################################### FLN ########################################
plt.sca(axs0[0])

logFLN = np.log(FLN)
masked_array = np.ma.array(logFLN, mask=np.isnan(logFLN))
cmap = matplotlib.cm.get_cmap("hot_r")
cmap.set_bad("black")
plt.imshow(logFLN, aspect="auto", cmap=cmap)
cbar = plt.colorbar()
cbar.set_label("", rotation=270)
plt.title("log(FLN)", fontsize=6)
plt.xticks(iter_, area_names, rotation=90, fontsize=4)
plt.yticks(iter_, area_names, fontsize=4)
plt.xlabel("From", fontsize=6)
plt.ylabel("To", fontsize=6)
cbar.ax.tick_params(labelsize=6)


################################### DISTANCE ########################################
plt.sca(axs0[1])
masked_array = np.ma.array(Distances, mask=np.isnan(Distances))
cmap = matplotlib.cm.get_cmap("copper")
cmap.set_bad("black")
plt.imshow(Distances, aspect="auto", cmap=cmap, vmin=0, vmax=50)
cbar = plt.colorbar()
cbar.set_label("", rotation=270, labelpad=10)
plt.xticks(iter_, area_names, rotation=90, fontsize=4)
plt.setp(axs0[1].get_yticklabels(), visible=False)
plt.yticks(iter_, area_names, fontsize=4)
axs0[1].yaxis.set_label([])
plt.xlabel("From", fontsize=6)
plt.title("Distances [mm]", fontsize=6)
cbar.ax.tick_params(labelsize=6)


################################### MODEL ########################################

def plot_png(ax, figname):
    png = plt.imread(figname)
    plt.sca(ax)
    im = plt.imshow(png, interpolation="none")
    plt.axis("off")
    pad = 0
    plt.xlim(-pad, png.shape[1] + pad)
    plt.ylim(png.shape[0] + pad, -pad)


for i in range(2):
    plot_png(axs1[i], f"figures/circuit{i + 1}.png")
    if i == 1:
        plt.title("Strong GBA", fontsize=6)
    else:
        plt.title("Weak GBA", fontsize=6)

################################### Firing rates ########################################
plt.sca(axs2[0])

plt.imshow(
    rates_avg_t.sel(gba="weak").sel(roi=sel_rois),
    aspect="auto",
    origin="lower",
    cmap="Greys",
    vmin=10,
    vmax=40,
)
plt.xticks(tidx, tticks, fontsize=6)
plt.yticks(range(rates_avg_f.sel(roi=sel_rois).sizes["roi"]), sel_rois, fontsize=6)
plt.xlabel("Time [s]", fontsize=6)

times = rates_avg_t.times.data
stim = np.zeros_like(times)
stim[np.logical_and(times >= 4500, times <= 5000)] = 1
plt.plot(stim * 2, "r")
plt.text(250, 2, "42 pA", fontsize=5)

plt.sca(axs2[1])

plt.imshow(
    rates_avg_t.sel(gba="strong").sel(roi=sel_rois),
    aspect="auto",
    origin="lower",
    cmap="Greys",
    vmin=10,
    vmax=40,
)
plt.xticks(tidx, tticks, fontsize=6)
plt.yticks(range(rates_avg_f.sel(roi=sel_rois).sizes["roi"]), sel_rois)
plt.xlabel("Time [s]", fontsize=6)
plt.setp(axs2[1].get_yticklabels(), visible=False)
plt.plot(stim, "r")
plt.text(250, 1, "22 pA", fontsize=5)


norm = matplotlib.colors.Normalize(vmin=10, vmax=40)
cmap = matplotlib.cm.get_cmap("Greys")

cbar = plt.colorbar(
    mappable=plt.cm.ScalarMappable(cmap=cmap, norm=norm),
    cax=axs2[2],
)

cbar.ax.set_ylabel("Firing rate [Hz]", fontsize=6, rotation=270, labelpad=14)
cbar.ax.tick_params(labelsize=6)


############################################ AIS #############################################"
plt.sca(axs3[0])
plt.imshow(
    AIS.isel(gba=1).sel(roi=sel_rois), aspect="auto", cmap="Greys", origin="lower",
    vmin=0, vmax=1.
)
plt.title("AIS", fontsize=6)
plt.xlabel("Time [s]", fontsize=6)
plt.xticks(tidx, tticks, fontsize=6)
plt.yticks(range(len(sel_rois)), sel_rois, fontsize=6)
#plt.setp(axs3[0].get_xticklabels(), visible=False)


norm = matplotlib.colors.Normalize(vmin=0, vmax=1.)
cmap = matplotlib.cm.get_cmap("Greys")

cbar = plt.colorbar(
    mappable=plt.cm.ScalarMappable(cmap=cmap, norm=norm),
    cax=axs3[1],
)
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_ylabel("", fontsize=6, rotation=270, labelpad=14)

############################################ TEff #############################################"
plt.sca(axs3[2])
plt.imshow(
    TEff_s.sel(roi=sel_rois_teff),
    aspect="auto",
    cmap="Greys",
    origin="lower",
    vmin=0,
    vmax=0.3,
)
# plt.yticks(range(TEff_s.sizes["roi"]), TEff_s.roi.data)
plt.title("TE from V1", fontsize=6)
plt.xticks(tidx, tticks, fontsize=6)
plt.xlabel("Time [s]", fontsize=6)
plt.yticks(range(len(sel_rois) - 1), sel_rois[1:], fontsize=6)
#plt.setp(axs3[2].get_xticklabels(), visible=False)


norm = matplotlib.colors.Normalize(vmin=0, vmax=0.3)
cmap = matplotlib.cm.get_cmap("Greys")

cbar = plt.colorbar(
    mappable=plt.cm.ScalarMappable(cmap=cmap, norm=norm),
    cax=axs3[3],
)
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_ylabel("", fontsize=6, rotation=270, labelpad=14)

############################################ TEfb #############################################"
plt.sca(axs3[4])
plt.imshow(
    TEfb_s.sel(roi=sel_rois_tefb),
    aspect="auto",
    cmap="Greys",
    origin="lower",
    vmin=0,
    vmax=0.3,
)
# plt.yticks(range(TEff_s.sizes["roi"]), TEff_s.roi.data)
plt.title("TE to V1", fontsize=6)
plt.xticks(tidx, tticks, fontsize=6)
plt.xlabel("Time [s]", fontsize=6)
plt.yticks(range(len(sel_rois) - 1), sel_rois[1:], fontsize=6)
#plt.setp(axs3[4].get_xticklabels(), visible=False)

norm = matplotlib.colors.Normalize(vmin=0, vmax=0.3)
cmap = matplotlib.cm.get_cmap("Greys")

cbar = plt.colorbar(
    mappable=plt.cm.ScalarMappable(cmap=cmap, norm=norm),
    cax=axs3[5],
)
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_ylabel("", fontsize=6, rotation=270, labelpad=14)

############################################ Syn #############################################"
plt.sca(axs3[6])
plt.imshow(
    pid_strong[4].sel(roi=sel_rois_syn),
    aspect="auto",
    cmap="Greys",
    origin="lower",
    vmin=0, vmax=.3
)
# plt.yticks(range(pid_weak[4].sizes["roi"]), pid_weak[2].roi.data)
plt.title("Synergy with V1", fontsize=6)
plt.xticks(tidx, tticks, fontsize=6)
plt.xlabel("Time [s]", fontsize=6)
plt.yticks(range(len(sel_rois) - 1), sel_rois[1:], fontsize=6)

norm = matplotlib.colors.Normalize(vmin=0, vmax=0.3)
cmap = matplotlib.cm.get_cmap("Greys")

cbar = plt.colorbar(
    mappable=plt.cm.ScalarMappable(cmap=cmap, norm=norm),
    cax=axs3[7],
)
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_ylabel("", fontsize=6, rotation=270, labelpad=14)
#plt.setp(axs3[6].get_xticklabels(), visible=False)

############################################ AIS polar #############################################"
plt.sca(axs4[0])
N = 29
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rates.roi.data), fontsize=6)

# Plot actual sales graph
plt.plot(theta, ais.sel(gba=0), "r")
plt.fill(theta, ais.sel(gba=0), "r", alpha=0.1)
plt.plot(theta, ais.sel(gba=1), "b")
plt.fill(theta, ais.sel(gba=1), "b", alpha=0.1)
plt.title("AIS", fontsize=6)

axs4[0].set_ylim([0, .75])
#axs4[0].set_rticks([0, 0.35, .75])
axs4[0].set_yticklabels(np.linspace(0, .75, 4), fontsize=6)

############################################ TEff polar #############################################
plt.sca(axs4[1])

N = 28
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

rois = rates.roi.data[1:]

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois), fontsize=6)


# Plot actual sales graph
plt.plot(theta, TEff_w_avg, "r")
plt.fill(theta, TEff_w_avg, "r", alpha=0.1)
plt.plot(theta, TEff_s_avg, "b")
plt.fill(theta, TEff_s_avg, "b", alpha=0.1)
plt.title("TE from V1", fontsize=6)

axs4[1].set_ylim([0, .1])
#axs4[1].set_rticks([0, 0.05, .1])
axs4[1].set_yticklabels(np.round(np.linspace(0, .035, 4), 3), fontsize=6)

############################################ TEfb polar #############################################
plt.sca(axs4[2])

N = 28
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25

rois = rates.roi.data[1:]

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois), fontsize=6)

# Plot actual sales graph
plt.plot(theta, TEfb_w_avg, "r")
plt.fill(theta, TEfb_w_avg, "r", alpha=0.1)
plt.plot(theta, TEfb_s_avg, "b")
plt.fill(theta, TEfb_s_avg, "b", alpha=0.1)
plt.title("TE to V1", fontsize=6)

axs4[2].set_ylim([0,.1])
#axs4[2].set_rticks([0, 0.05, .1], fontsize=6)
axs4[2].set_yticklabels(np.round(np.linspace(0, .035, 4), 3), fontsize=6)

############################################ SYn polar #############################################
plt.sca(axs4[3])
syn_w = pid_weak[-1].mean("times")
syn_s = pid_strong[-1].mean("times")

# Compute pie slices
N = 28
theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
width = 0.25
rois = rates.roi.data[1:]

lines, labels = plt.thetagrids(theta * 360 / (2 * np.pi), (rois), fontsize=6)


# Plot actual sales graph
plt.plot(theta, syn_w, "r", label="weak GBA")
plt.fill(theta, syn_w, "r", alpha=0.1)
plt.plot(theta, syn_s, "b", label="strong GBA")
plt.fill(theta, syn_s, "b", alpha=0.1)
plt.title("Synergy", fontsize=6)

axs4[3].set_ylim([0,.1])
#axs4[3].set_rticks([0, .1], fontsize=6)
axs4[3].set_yticklabels(np.round(np.linspace(0, .1, 4), 2), fontsize=6)


axs4[3].legend(
    loc="center left",
    bbox_to_anchor=(-2.5, -0.4),
    ncol=2,
    fontsize=8,
    frameon=False,
)

############################################ Hierarchy relation #############################################
plt.sca(axs5[0])

rV1z = (rV1 - rV1.mean("times")) / rV1.std("times")
r24cz = (r24c - r24c.mean("times")) / r24c.std("times") + 4
synz = (syn - syn.mean("times")) / syn.std("times") + 8

rV1z = rV1z.assign_coords({"times": (rV1z.times.data - 4500) / 1000})
r24cz = r24cz.assign_coords({"times": (r24cz.times.data - 4500) / 1000})
synz = synz.assign_coords({"times": (synz.times.data - 4500) / 1000})

rV1z.plot(ax=axs5[0])
r24cz.plot(ax=axs5[0])
synz.plot(ax=axs5[0], c="g")

plt.vlines(rV1z.times[rV1z.argmax()].data, -1, 12, color="b", ls="--")
plt.vlines(r24cz.times[r24cz.argmax()].data, -1, 12, color="orange", ls="--")
plt.vlines(synz.times[synz.argmax()].data, -1, 12, color="green", ls="--")

plt.text(-1.7, 0, "V1")
plt.text(-1.7, 3.5, "46d")
plt.text(-1.7, 8.2, "syn(V1, 46d)")

plt.yticks([])
plt.xticks(tticks, tticks, fontsize=6)
plt.xlabel("Time [s]", fontsize=6)
#axs5[0].legend(["rate V1", "4 * rate 46d", "350 * syn"], frameon=False, fontsize=8)
[axs5[0].spines[key].set_visible(False) for key in ["top", "right", "left"]]


tidx, tticks = get_time_idx(rV1z.times.data)


#plt.xticks(tidx, tticks, fontsize=6)
plt.yticks(fontsize=6)
plt.ylabel("")
plt.title("")

plt.sca(axs5[1])
x = Distances[0, 1:]
y = np.abs(tau1[1:].data - tau2.data) * 0.1
x = x[rois != "2"]
y = y[rois != "2"]
sns.regplot(x=x, y=y, n_boot=100, ci=None)
# for i, r in enumerate(rois[rois != "2"]):
#    plt.text(x[i], y[i], r, fontsize=6)

plt.ylabel(r"$|t_r - t_{syn}|$", fontsize=6)
plt.xlabel(" Distance from V1 [mm]", fontsize=6)
[axs5[1].spines[key].set_visible(False) for key in ["top", "right"]]
r = np.corrcoef(x, y)[0, 1]
# axs5[1].legend([f"R = {r:.2f}"], frameon=False, fontsize=8)
plt.xticks(fontsize=6)
plt.yticks(fontsize=6)
plt.text(40, 20, f"R = {r:.2f}")


################################################## SSS ##############################################################
plt.sca(axs6[0])
AISsel = AIS.sel(gba=1, roi="46d")

AISsel = AISsel.assign_coords({"times": SSSs.times.data})

AISsel.plot(ax=axs6[0], x="times", label="AIS")
SSSs.sel(roi="46d").plot(ax=axs6[0], x="times", label="stim. specific AIS")

tidx, tticks = get_time_idx(SSSs.times.data)
plt.xticks(tticks, tticks, fontsize=6)
plt.yticks(fontsize=6)

plt.xlabel("Time [s]", fontsize=6)
#plt.ylabel("Stimulus specific storage", fontsize=6)
plt.title("area 46d", fontsize=8, pad=15)
[axs6[0].spines[key].set_visible(False) for key in ["top", "right"]]

axs6[0].legend(
    loc="upper center",
    bbox_to_anchor=(.5, 1.1),
    ncol=2,
    fontsize=8,
    frameon=False,
)

bg = plot.Background(visible=False)

plt.savefig("figures/panel_wang_ipp.pdf")

In [ ]:
tticks

In [ ]:
tidx, tticks = get_time_idx(SSSs.times.data)


In [ ]:
tidx